# Импорты

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go

In [ ]:
data = pd.read_csv('DATA\dst-3.0_16_1_hh_database.csv', sep=';')
ex_rates = pd.read_csv('DATA\ExchangeRates.csv')

# Исследование структуры данных

##### 2.1

In [ ]:
display(data.shape)

##### 2.2

In [ ]:
display(data.head(5), data.tail(5))

##### 2.3

In [ ]:
data.info()

##### 2.4

In [ ]:
cols_null = (data.isnull().mean() * 100).sort_values(ascending=False)
display(cols_null)

##### 2.5

In [ ]:
display(data['Опыт работы'].nunique(),
        data['Последняя/нынешняя должность'].mode())

In [ ]:
data_hh = data.copy()

# Предобработка данных

#### Преобразование признака Образования и Вуза

In [ ]:
data['Образование и ВУЗ'].apply(lambda x:' '.join(x.split()[:x.split().index('образование')])).unique()

In [ ]:
data_hh['Образование и ВУЗ'] = data_hh['Образование и ВУЗ'].apply(lambda x: ' '.join(x.split()[:x.split().index('образование')]))

##### 3.3

In [ ]:
data_hh['Образование и ВУЗ'].value_counts()

In [ ]:
data_hh['Пол'] = data_hh['Пол, возраст'].apply(lambda x: x.split()[0][0])
data_hh['Возраст'] = data_hh['Пол, возраст'].apply(lambda x: int(x.split(',')[1].split()[0]))

##### 3.2

In [ ]:
display(data_hh['Пол'].value_counts(normalize=True) * 100,
        data_hh['Возраст'].mean())

In [ ]:
data_hh.drop(axis=1, labels='Пол, возраст', inplace=True)


#### Преобразовование признака опыта работы

In [ ]:
def apply_exp(x):
    if x is np.nan or x == 'Не указано':
        return None
    
    m,y = (0,0)
    
    x = x.split()
    
    if 'года' in x[:7] and x.index('года')<=7:
        y = int(x[x.index('года')-1])
    elif 'лет' in x[:7] and x.index('лет')<=7:
        y = int(x[x.index('лет')-1])
    elif 'год' in x[:7] and x.index('год')<=7:
        y = int(x[x.index('год')-1])
        print(x[:7])
    
    
    if 'месяцев' in x[:7] and x.index('месяцев')<=7:
        m = int(x[x.index('месяцев')-1])
    elif 'месяца' in x[:7] and x.index('месяца')<=7:
        m = int(x[x.index('месяца')-1]) 
    elif 'месяц' in x[:7] and x.index('месяц')<=7:
        m = 1
        

    return y*12 + m

data_hh['Опыт работы'] = data_hh['Опыт работы'].apply(apply_exp)
# data_hh['Опыт работы'][data_hh['Опыт работы'].isnull()]
display(data_hh['Опыт работы'].median())
# data_hh['Опыт работы'].max()

#### Гистограмма и коробчатая диаграмма для опыта работы

In [ ]:
fig1 = plt.figure(figsize=(10, 7))
gist = sns.histplot(
    data=data_hh,
    x = 'Опыт работы'
)
gist.axvline(data_hh['Опыт работы'].median(),color = 'k')
gist.axvline(data_hh['Опыт работы'].mean(),ls='--',color = 'k')
fig = plt.figure(figsize=(20, 7))
boxplot = sns.boxplot(
    data=data_hh,
    x = 'Опыт работы',
    width=0.9
)
boxplot.set_title('Опыт работы') 

эта часть кода, нужна, чтоб понять какие варианты есть в таблице в пункте про переезд

In [ ]:
def finder(x):
    x = x.split(' , ')
    
    for i in range(len(x)):
        # if not (x[i].find('готов') == -1) and not(x[i].find('ком') == -1):
        #     return x[i].split(' (')[0]
        if not (x[i].find('не хочу') == -1) and not(x[i].find('пере') == -1):
            return x[i].split(' (')[0]
# data_hh['Город, переезд, командировки'].apply(lambda x: x[x.split(' , ')[:20].split() == 'готов'])
print(list(data_hh['Город, переезд, командировки'].apply(finder).unique()))

#### Преобразовавание признака «Город, переезд, командировки»

In [ ]:
def city(x):
    million_cities = ['Новосибирск', 'Екатеринбург', 'Нижний Новгород', 'Казань', 'Челябинск', 'Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск', 'Пермь', 'Воронеж', 'Волгоград' ]
    x = x.split(' , ')
    city = None
    
    
    if x[0] == 'Москва' or x[0] == 'Санкт-Петербург':
        city = x[0]
    elif x[0] in million_cities:
        city = 'город-миллионник'
    else:
        city = 'другие'
    
    return city

def crossing(x):
    x = x.split(' , ')
    crossing = None
    
    for i in range(len(x)):   
        if not (x[i].find('не готов') == -1) and not(x[i].find('переезд') == -1):
            crossing = False
        elif not (x[i].find('готов') == -1) and not(x[i].find('переезд') == -1):
            crossing = True
        elif not (x[i].find('хоч') == -1) and not(x[i].find('пере') == -1):
            crossing = True
    
    return crossing

def business_trip(x):
    x = x.split(' , ')
    business_trip = False
    
    for i in range(len(x)):   
        if not (x[i].find('не готов') == -1) and (not(x[i].find('ком') == -1)):
            business_trip = False
        elif not (x[i].find('готов') == -1) and not(x[i].find('ком') == -1):
            business_trip = True
    
    return business_trip

data_hh['Город'] = data_hh['Город, переезд, командировки'].apply(city)
data_hh['Готовность к переезду'] = data_hh['Город, переезд, командировки'].apply(crossing)
data_hh['Готовность к командировкам'] = data_hh['Город, переезд, командировки'].apply(business_trip)
data_hh.drop(axis=1, labels='Город, переезд, командировки', inplace=True)

##### 3.4

In [ ]:
display(data_hh['Город'].value_counts(normalize=True))
print((data_hh[(data_hh['Готовность к переезду'] == data_hh['Готовность к командировкам']) & (data_hh['Готовность к командировкам'] == True) ].shape[0] / data_hh.shape[0] * 100))


Проверяем какие есть случаи желаемой занятости и графика

In [ ]:
l_range = []
def f(x):
    x = x.split(', ')
    global l_range
    for i in range(len(x)):
        if x[i] not in l_range:
            l_range.append(x[i])
            

data_hh['Занятость'].apply(f)
emp = l_range
l_range = []
data_hh['График'].apply(f)
charts = l_range
print(l_range)

##### One Hot Encoding 3.5

In [ ]:
for i in emp:
    data_hh[i] = data_hh['Занятость'].apply(lambda x: True if not x.find(i) == -1 else False)
for i in charts:
    data_hh[i] = data_hh['График'].apply(lambda x: True if not x.find(i) == -1 else False)
data_hh = data_hh.drop('Занятость', axis=1)
data_hh = data_hh.drop('График', axis=1)    

Предыдущий вариант этой предабработки

In [ ]:
# def employment(x):
#     global a1, a2, a3, a4, a5
#     x = x.split(', ')
#     a1.append(False)
#     a2.append(False)
#     a3.append(False)
#     a4.append(False)
#     a5.append(False)
#     # print(x)
#     for i in range(len(x)):
#         if x[i] == 'частичная занятость':
#             a1[-1] = True
#         if x[i] == 'проектная работа':
#             a2[-1] = True
#         if x[i] == 'полная занятость':
#             a3[-1] = True
#         if x[i] == 'стажировка':
#             a4[-1] = True
#         if x[i] == 'волонтерство':
#             a5[-1] = True

In [ ]:
# data_hh['Занятость'].apply(employment)

In [ ]:
# data_hh['частичная занятость'] = a1
# data_hh['проектная работа'] = a2
# data_hh['полная занятость'] = a3
# data_hh['стажировка'] = a4
# data_hh['волонтерство'] = a5
# # data_hh['Занятость'] = data_hh['Занятость'].apply(lambda x:''.join(x.split(', ')))
# data_hh

##### 3.5

In [ ]:
display(data_hh[data_hh['проектная работа'] & data_hh['волонтерство']].shape[0],
        data_hh[data_hh['вахтовый метод'] & data_hh['гибкий график']].shape[0])

#### merge таблицы с валютами

переводим в тип данных datetime

In [ ]:
data_hh['Обновление резюме'] = data_hh['Обновление резюме'].apply(lambda x: pd.to_datetime(x.split()[0], dayfirst=True))

##### Переименование валют в таблице с ее названиями


In [ ]:
ISO_dict = {
    'грн.':"UAH",
    'USD':'USD',
    'EUR':'EUR',
    'сум':'UZS',
    'бел.руб.':'BYN',
    'KGS':'KGS',
    'AZN':'AZN',
    'KZT':'KZT',
    'руб.':'RUB'
}

data_hh['ЗП'] = data['ЗП'].apply(lambda x: x.split()[0])
data_hh['ISO'] = data['ЗП'].apply(lambda x: x.split()[1])
data_hh['ISO'] = data_hh['ISO'].apply(lambda x: ISO_dict[x])

Проверка количества значений

In [ ]:
data_hh.ISO.unique()

##### сам merge таблицы и удаление посторонних столбцов

In [ ]:
ex_rates = ex_rates.drop('time', axis=1)
ex_rates = ex_rates.drop('vol', axis=1)
ex_rates = ex_rates.drop('per', axis=1)
ex_rates.date = pd.to_datetime(ex_rates.date)
data_hh = data_hh.merge(ex_rates, how='left', left_on=['ISO', 'Обновление резюме'], right_on=['currency','date'])
data_hh = data_hh.drop('date', axis=1)
data_hh = data_hh.drop('currency', axis=1)
data_hh.loc[data_hh['ISO']=='RUB','close'] = 1
data_hh.loc[data_hh['ISO']=='RUB','proportion'] = 1
data_hh['ЗП'] = data_hh['ЗП'].astype('float64')
data_hh['ЗП (руб)'] = data_hh['ЗП'] * data_hh['close'] / data_hh['proportion']
# data_hh['Образование'] = data_hh['Образование и ВУЗ']
# data_hh = data_hh.drop('Образование и ВУЗ', axis=1)
data_hh = data_hh.drop('proportion', axis=1)
data_hh = data_hh.drop('close', axis=1)
data_hh = data_hh.drop('ЗП', axis=1)
data_hh = data_hh.drop('ISO', axis=1)

##### 3.6

In [ ]:
data_hh['ЗП (руб)'].median()

# EDA и визуализация

##### 4.1 и график

In [ ]:
age_gis = px.histogram(
    data_frame=data_hh,
    height=300,
    width=900,
    x = 'Возраст',
    marginal='box'
)
age_gis_dist_data = list(data_hh['Возраст'])
age_gis.add_vline(x = float(data_hh['Возраст'].mode()), line_color = 'Cyan', line_dash = 'dash')
age_gis_dist = ff.create_distplot([age_gis_dist_data],['Возраст'],curve_type='kde', show_hist=True)
age_gis.show()

не запусать код ниже, если лаги не желательны (гистограмма с __kde__)

In [ ]:

age_gis_dist.show()


*Мода* распределния лежит рядом с *медианой* и равна __30__.
Минимум признака равен __15__ годам, а максимум __100__, что скорее всего является выбросом.
Понятно, что большинтво наблюдений будут находиться на межквартильном расстоянии, с __25__ по __75__. Аномалией можно считать наблюдение в 100 лет - очень мала вероятность встретить такого соискателя

In [ ]:
exp_gis_data = data_hh[~ data_hh['Опыт работы'].isnull()]
exp_gis = px.histogram(
    data_frame= exp_gis_data,
    height=300,
    width=900,
    x = 'Опыт работы',
    marginal='box',
    nbins=200
)
exp_gis.add_vline(x = float(exp_gis_data['Опыт работы'].mode()), line_color = 'Cyan', line_dash = 'dash')
# exp_gis.add_vline(x = exp_gis_data['Опыт работы'].mode(), line_color = 'Red')
exp_gis_dist_data = list(exp_gis_data['Опыт работы'])
exp_gis_dist = ff.create_distplot([exp_gis_dist_data],['Опыт работы'],curve_type='kde', show_hist=True)
exp_gis.show()
exp_gis_data[exp_gis_data['Опыт работы'] == 1188]



Как мы видим, мода распределения лежит в одной корзине с 9 другими значениями, начертим линию - видно, что *мода* равна __81__ месяцу. Минимум признака равен __1__, а максимум __1188__. Большинство значений лежат в межквартильном расстоянии, то есть между __57__ и __154__ месяцами. Аномалией конечно является наблюдение в 1188 месяцов(99 лет), которое принадлежит человеку с возрастом в 48 лет.

не запусать код ниже, если лаги не желательны (гистограмма с __kde__)

In [ ]:
exp_gis_dist.show()

##### 4.3

In [ ]:
salary_gis_data = data_hh[~ data_hh['ЗП (руб)'].isnull()]
salary_gis_data.to_csv('DATA/exp.csv')
# salary_gis_data.drop(salary_gis_data['ЗП (руб)'].idxmax(), axis=0, inplace=True)

salary_gis = px.histogram(
    data_frame=salary_gis_data,
    height=300,
    width=1200,
    histnorm='percent',
    # log_x=True,
    x = 'ЗП (руб)',
    marginal='box',
)
salary_gis.add_vline(x = float(salary_gis_data['ЗП (руб)'].mode()), line_color = 'Cyan', line_dash = 'dash')

По графику довольно трудно заметить, но *мода* распределения равна __50000__. *Минимум* равен 1, а *максимум* __24__ __миллионам__. Большинство зароботных плат лежит в межкваритльном рассотянии, то есть с __37054__ до __95000__. Аномалией для данного признака можно считать его хвост а также огромные значения, типа 1750000, 2500000, 3 миллионам, 7675224 и максимума признака, то есть все 5 значений превышающих миллион

In [ ]:
# salary_gis_data_exp = data_hh[~ data_hh['ЗП (руб)'].isnull()]
# # salary_gis_data.drop(salary_gis_data['ЗП (руб)'].idxmax(), axis=0, inplace=True)
# salary_gis_data_exp['ЗП (руб)'] = np.log(salary_gis_data_exp['ЗП (руб)']+1)
# salary_gis = px.histogram(
#     data_frame=salary_gis_data_exp,
#     height=300,
#     width=800,
#     x = 'ЗП (руб)',
#     marginal='box',
#     nbins = 1000
# )
# salary_gis.show()
# # salary_gis.add_vline(x = float(salary_gis_data['ЗП (руб)'].mode()), line_color = 'Cyan', line_dash = 'dash')

#вторая версия 
# salary_gis_data_exp = data_hh[~ data_hh['ЗП (руб)'].isnull()]
# # salary_gis_data.drop(salary_gis_data['ЗП (руб)'].idxmax(), axis=0, inplace=True)
# # salary_gis = px.histogram(
# #     data_frame=salary_gis_data_exp,
# #     height=300,
# #     width=800,
# #     x = 'ЗП (руб)',
# #     marginal='box',
# #     nbins = 1000
# # )
# salarty_gis = go.Figure()
# salarty_gis.add_trace(go.Histogram(x =salary_gis_data_exp['ЗП (руб)']))
# salarty_gis.update_xaxes(type = 'log')
# salarty_gis.show()
# # salary_gis.add_vline(x = float(salary_gis_data['ЗП (руб)'].mode()), line_color = 'Cyan', line_dash = 'dash')

##### 4.4

In [ ]:
salary_educ_data = data_hh[data_hh['ЗП (руб)'] < 1000000]
salary_educ= px.box(
    data_frame=salary_educ_data,
    log_x=True,
    x='ЗП (руб)',
    color = 'Образование и ВУЗ',
    height=400,
    width=600
)
salary_educ.add_vline(x=salary_educ_data.loc[salary_educ_data['Образование и ВУЗ'] == 'Среднее', 'ЗП (руб)'].median(), line_color = '#ab63fa',line_dash = 'dash') #фиолетовый   
salary_educ.add_vline(x=salary_educ_data.loc[salary_educ_data['Образование и ВУЗ'] == 'Среднее специальное', 'ЗП (руб)'].median(), line_color = '#00cc96') #изумрудный
salary_educ.add_vline(x=salary_educ_data.loc[salary_educ_data['Образование и ВУЗ'] == 'Высшее', 'ЗП (руб)'].median(), line_color = '#ef553b', line_dash = 'dash') #красный
salary_educ.add_vline(x=salary_educ_data.loc[salary_educ_data['Образование и ВУЗ'] == 'Неоконченное высшее', 'ЗП (руб)'].median(), line_color = '#636efa', line_dash = 'dash')#синий


salary_educ.show()

Для среденго специального образования характерен *минимум* желаемой зарботной платы, в противу высшему, для которого характерен *максимум*.
Скорее всего при прогнозировании зароботной платы также стоит смотреть и на образование человека, оставившего анкету, ведь видна прямая корреляция зароботной платы и роста уровня образования

##### 4.5

In [ ]:
salary_educ_data = data_hh[data_hh['ЗП (руб)'] < 1000000]
salary_city= px.box(
    data_frame=salary_educ_data,
    log_x=True,
    x='ЗП (руб)',
    color = 'Город',
    height=400,
    width=600
)
salary_city.add_vline(x=salary_educ_data.loc[salary_educ_data['Город'] == 'Санкт-Петербург', 'ЗП (руб)'].median(), line_color = '#ab63fa',line_dash = 'dash') #фиолетовый   
salary_city.add_vline(x=salary_educ_data.loc[salary_educ_data['Город'] == 'город-миллионник', 'ЗП (руб)'].median(), line_color = '#00cc96') #изумрудный
salary_city.add_vline(x=salary_educ_data.loc[salary_educ_data['Город'] == 'Москва', 'ЗП (руб)'].median(), line_color = '#ef553b', line_dash = 'dash') #красный
salary_city.add_vline(x=salary_educ_data.loc[salary_educ_data['Город'] == 'другие', 'ЗП (руб)'].median(), line_color = '#636efa', line_dash = 'dash')#синий

salary_city.show()

Самая высокая желаемая медианная зароботная плата, очевидно в Москве, наименьшую медианную делят категории: Город-миллионник и другие. Скорее всего при прогнозировании этот признак будет невероятно важен, ведь на данных можно легко заметить корреляцию между желаемой ЗП и местоположением человека, подавшего анкету

##### 4.6

In [ ]:
# display(data_hh.groupby(['Готовность к переезду','Готовность к командировкам'], as_index=False)['ЗП (руб)'].median())
salary_much_bar = px.bar(
    data_frame=data_hh.groupby(['Готовность к переезду','Готовность к командировкам'], as_index=False)['ЗП (руб)'].median(), 
    y = 'ЗП (руб)',
    x = 'Готовность к переезду',
    color = 'Готовность к командировкам',
    barmode='group',
    height=600,
    width=1000
)
salary_much_bar.show()

По графику, можно легко заметить, что на чем большие пункты таблицы соискатель готов пойти, тем большую заработную плату он требует. Это правило на наших данных работает безотказно.

##### 4.7

In [ ]:
salary_age_educ_data = data_hh.pivot_table(values='ЗП (руб)', columns = 'Возраст', index = 'Образование и ВУЗ', aggfunc='median' )
heat_salary = px.imshow(
    salary_age_educ_data,
    labels = dict(x='Возраст', y='Образование'),
    width=1000,
    height=600
)
heat_salary.show()

На всех 4 частях тепловой карты видна темная область вначале - людям нужно время , чтобы получить образование. Самый быстрый росто свойственен высшему образованию. В росте в зависимости от возраста в высшем образовании можно увидеть два пика, один в 38 лет, другой в 76, который скорее всего связан с выбром в данных. Такой же аномальный резкий скачак можно увидет и в среднем специальном в 67 годах.

##### 4.8

In [ ]:
scatterplot_exp_data = data_hh.copy()
scatterplot_exp_data['Опыт работы'] =  data_hh['Опыт работы'] / 12
scatterplot_exp = px.scatter(
    data_frame=scatterplot_exp_data,
    x = 'Возраст',
    y = 'Опыт работы',
    height=600,
    width=600
)

scatterplot_exp.add_trace(
    go.Scatter(
        x = [0,100],
        y = [0,100],
        mode='lines',
        line=go.scatter.Line(color='black'),
        showlegend=True)
)
scatterplot_exp.show()

По графику легко можно определить количество точных выбросов в признаке возраста и опыта работы. Все точки, лежащие выше или на прямой - выбросы, их найдено 7.

##### 4.9

In [ ]:
exp_age_salary = px.scatter_3d(
    data_frame=data_hh[data_hh['ЗП (руб)']<1000000],
    x='Возраст',
    y='Опыт работы',
    z='ЗП (руб)',
    log_x = True,
    log_y = True,
    width=1000,
    height=900,
    
)

exp_age_salary.show()

По графику легко заметить, что пик зароботной платы наблюдается в возрасте 30 лет со стажем примерно в 10. Также можно подтвердить довольно очевидный вывод, что опыт работы растет с возрастом соискателя

##### 4.10

In [ ]:
gender_age_salary = px.scatter(
    data_frame=data_hh[data_hh['ЗП (руб)']<1000000],
    color = 'Пол',
    y = 'ЗП (руб)',
    x = 'Опыт работы'
)

gender_age_salary.show()

Как видно из данного графика, в среднем ЗП и опыт работы у мужчин среди соискателей больше, также можно заметить, что мужчины и требуют большей зароботной платы чем слабый пол.

# Очистка данных

##### 5.1

In [ ]:
mask = data_hh.duplicated()
print(data_hh[mask].shape[0])
data_hh = data_hh.drop_duplicates()


##### 5.2

In [ ]:
display(data_hh.info())

In [ ]:
data_hh = data_hh[~data_hh['Последнее/нынешнее место работы'].isnull()]
data_hh = data_hh[~data_hh['Последняя/нынешняя должность'].isnull()]
data_hh = data_hh.fillna({'Опыт работы':data_hh['Опыт работы'].median()})


##### 5.3

In [ ]:
data_hh['Опыт работы'].mean()

##### 5.4

In [ ]:
data_hh = data_hh[(1000<=data_hh['ЗП (руб)'])& (data_hh['ЗП (руб)']<=1000000)]


##### 5.5

In [ ]:
data_hh = data_hh[(data_hh['Возраст']) > (data_hh['Опыт работы']/12)]

In [ ]:
data_hh['Возраст'] = data_hh['Возраст'].astype('uint8')

### 5.6

In [ ]:
data_hh_log= np.log(data_hh['Возраст'].astype('float64')+1)

In [ ]:
age_gis_log = px.histogram(
    data_frame=data_hh_log,
    height=300,
    width=900,
    # x = 'Возраст',
    nbins=91,
    marginal='box'         
)
print(np.log(data_hh['Возраст'].mean()))
display(data_hh_log)
print(data_hh_log.astype('float64').mean())
age_gis_log.add_vline(data_hh_log.mean(),line_color = '#635efa' ,line_dash = 'dash')
display(data_hh_log.skew())
age_gis_log.add_vrect(x0=data_hh_log.mean() + 4 * data_hh_log.std(), 
                      x1=data_hh_log.mean() - 3 * data_hh_log.std(),
                      annotation_text = 'Данные, подходящие           под метод трех сигм                                      ',
                      fillcolor = 'skyblue',
                      opacity = 0.25)

# age_gis_log.add_vline()
age_gis_log.show()


Как можно понять из метода skew, распределение получилось с правосторонней асимметрией.

In [ ]:
def outliers_z_score_mod(data, feature,left = 3, right = 3, log_scale=False):
    if log_scale:
        x = np.log(data[feature].astype('float64')+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned


outliers, cleaned = outliers_z_score_mod(data_hh, 'Возраст',left = 3,right=4, log_scale=True)

In [ ]:
data_hh = cleaned.copy()

#  Окончательные данные 


In [ ]:
data_hh.to_csv('DATA/data_hh.csv')